# UNINA-YOLO-DLA: Colab Dry Run

Versione robusta per il dry run.
Include ricerca automatica della root del repository.

In [ ]:
# @title 1. Setup Repository & Environment
import os
import sys

# 1. CLONE
# Proviamo a clonare se non troviamo cartelle che sembrano il repo
potential_names = ['unina-yolo-dla', 'UNINA-YOLO-DLA']
found_repo = any(os.path.exists(n) for n in potential_names)

if not found_repo and not os.path.exists('requirements.txt'):
    print("Cloning repository...")
    !git clone https://github.com/raider99k/unina-yolo-dla.git

# 2. NAVIGATE TO ROOT
# Cerchiamo requirements.txt per capire dove siamo
target_file = 'requirements.txt'
root_dir = None

# Check current dir
if os.path.exists(target_file):
    root_dir = os.getcwd()
else:
    # Check subdirs
    for root, dirs, files in os.walk('.'):
        if target_file in files:
            root_dir = root
            break

if root_dir:
    os.chdir(root_dir)
    print(f"Found root directory: {os.getcwd()}")
    print("Pulling latest changes...")
    !git pull
else:
    print("ERROR: Could not find requirements.txt anywhere. Checks executed:")
    print(f"Current dir: {os.getcwd()}")
    print(f"Directory content: {os.listdir('.')}")
    raise FileNotFoundError("requirements.txt not found. Repository clone failed or structure unexpected.")

# 3. FIX COLAB RAPIDS CONFLICT (CUDA 13)
print("Cleaning up conflicting RAPIDS libraries...")
%pip uninstall -y cudf-cu12 cuml-cu12 cugraph-cu12 cuspatial-cu12 raft-cu12 pylibraft-cu12 rmm-cu12

# 4. INSTALL DEPENDENCIES
print("Installing dependencies...")
%pip install sphinx sphinx-glpi-theme
%pip install --no-deps --index-url https://pypi.ngc.nvidia.com pytorch-quantization==2.1.2
%pip install -r requirements.txt

In [ ]:
# @title 2. Generate Synthetic Data
!python tools/generate_synthetic_fsd.py --num-train 100 --num-val 20 --seed 42

In [ ]:
# @title 3. Run Training (Dry Run)
# Removed --name argument as train.py does not support it
!python train.py --data datasets/fsd_synth/fsd_synth.yaml --epochs 3 --batch 8 --imgsz 640 --device 0 --project runs/colab_debug